In [456]:
%matplotlib inline
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from scipy import stats
from ast import literal_eval # 문자열 모형의 딕트를 스근하게 딕트로 바꾸어 준다. 
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from surprise import Reader, Dataset, SVD, evaluate


In [457]:
dataSet = pd.read_csv('crawler/top.csv')
dataSet = dataSet.drop(columns='Unnamed: 0', axis=1)
dataSet2 = dataSet.copy()
reviewData = dataSet[['cosmeticId', 'type', 'review']]


In [458]:
reviewArray = ['' for i in range(100)]
for index, row in reviewData.iterrows():
    reviewArray[row['cosmeticId']] += row['review']

In [459]:
name_popId = dataSet[['name', 'cosmeticId']]
id2name = dataSet[['cosmeticId', 'name']]
name_popId = name_popId.drop_duplicates().set_index('name')
id2name = id2name.drop_duplicates().set_index('cosmeticId')

In [460]:
dataSet = dataSet.drop('name', axis=1)

In [461]:
dataSet.head(5)
seriesReview = pd.Series(reviewArray)

In [462]:
tf = TfidfVectorizer(analyzer='word', min_df=2, stop_words=['\r', '\n'], sublinear_tf=True)
tf_matrix = tf.fit_transform(seriesReview)

In [463]:
tf_matrix

<100x4970 sparse matrix of type '<class 'numpy.float64'>'
	with 31625 stored elements in Compressed Sparse Row format>

In [464]:
cosine_sim = linear_kernel(tf_matrix, tf_matrix)
cosine_sim[0]
reader = Reader() 

In [465]:
dataSet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1923 entries, 0 to 1922
Data columns (total 4 columns):
cosmeticId    1923 non-null int64
rate          1923 non-null float64
review        1923 non-null object
type          1923 non-null object
dtypes: float64(1), int64(1), object(2)
memory usage: 60.2+ KB


In [466]:
type = {'건성': 0, '지성': 1, '중성': 2, '복합성': 3, '민감성': 4}
dataSet['type'] = dataSet['type'].map(type)
trainData = dataSet[['type', 'cosmeticId', 'rate']].drop_duplicates()
data = Dataset.load_from_df(trainData , reader)

In [467]:
data.split(n_folds=5)

In [468]:
svd = SVD()
evaluate(svd, data, measures=['RMSE', 'MAE'])

Evaluating RMSE, MAE of algorithm SVD.

------------
Fold 1
RMSE: 0.2943
MAE:  0.2124
------------
Fold 2
RMSE: 0.3651
MAE:  0.2399
------------
Fold 3


C:\Users\LeeSangHoon\Anaconda3\envs\tensorflow\lib\site-packages\surprise\evaluate.py:66: UserWarning: The evaluate() method is deprecated. Please use model_selection.cross_validate() instead.
  'model_selection.cross_validate() instead.', UserWarning)
C:\Users\LeeSangHoon\Anaconda3\envs\tensorflow\lib\site-packages\surprise\dataset.py:193: UserWarning: Using data.split() or using load_from_folds() without using a CV iterator is now deprecated. 
  UserWarning)


RMSE: 0.3035
MAE:  0.2328
------------
Fold 4
RMSE: 0.2930
MAE:  0.2202
------------
Fold 5
RMSE: 0.3246


MAE:  0.2244
------------
------------
Mean RMSE: 0.3161
Mean MAE : 0.2259
------------
------------


In [469]:
trainset = data.build_full_trainset()
svd.fit(trainset)


In [470]:
def type_cosmeticsNum(type, cosmeticName=''):
    if cosmeticName != '':
        idx = name_popId['cosmeticId'][cosmeticName]
        print(idx)
    else:
        sortRate = dataSet[dataSet['type'] == type].drop_duplicates(['cosmeticId'])
        sortRate = sortRate.sort_values('rate', ascending=False)
        idx = sortRate['cosmeticId'][:10]
        sortRate = sortRate.reset_index(drop=True)
        sortRate['lenReview'] = 0
        for i, k in enumerate(idx):
            sortRate.loc[i, 'lenReview'] = len(dataSet2[dataSet2['cosmeticId'] == k]['review'])
        sortRate = sortRate.sort_values('lenReview', ascending=False)
        idx = sortRate['cosmeticId'][:1]
    sim_scores = list(enumerate(cosine_sim[int(idx)]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:26]
    cosmetic_indcies = [i[0] for i in sim_scores]
    cosmeticpd = pd.Series(cosmetic_indcies)

    # #cosmetics = dataSet.iloc[cosmetic_indcies][['cosmeticId', 'rate', 'review', 'type']]
    cosmeticpd = {'cosmeticId': cosmetic_indcies, 'est': cosmeticpd.apply(lambda x: svd.predict(type, x).est
                                                                          )}
    cosmeticpd = pd.DataFrame(cosmeticpd, columns=['cosmeticId', 'est'])
    cosmeticpd = cosmeticpd.sort_values('est', ascending=False)

    # finalCosmetics = cosmetics.drop_duplicates(['cosmeticId'])
    # finalCosmetics = finalCosmetics['cosmeticId'].reset_index(drop=True)
    # preCosmeticName = name_popId.index[finalCosmetics]
    for id in cosmeticpd['cosmeticId']:
        print(id2name.loc[id]['name'])


In [475]:
type_cosmeticsNum(1, '비타민 워터리 선 젤 SPF50+ PA++++')

0
UV 아쿠아 리치 워터리 젤 SPF 50+ PA++++
마일드 프로텍트 촉촉한 선크림 SPF30 PA++
퓨어 미네랄 선크림 SPF50+ PA++++
내추럴 미네랄 썬 프로텍션
슈퍼푸드_블루베리 리밸런싱 워터리 선크림 SPF37 PA+++
워터풀 썬 업 크림  SPF50+ PA++++
익스트림 모이스처 선크림 SPF50+ PA++++
산소수 라이트 선 젤 SPF50+ PA++++
마일드-S 모이스쳐 라이트 선 젤 SPF50+ PA++++
스킨 디펜스 멀티 프로텍션 에센스 SPF50+ PA+++ 
UV 퍼펙트 에어쿨 선 젤 SPF50+ PA++++
올 어라운드 세이프 블록 에센스 선밀크 SPF50+ PA+++ 
NEW 이데베논 선블럭 SPF50+PA++++
마일드 프로텍트 에센스 선 SPF50+ PA++++
모이스쳐라이징 썬 스크린 SPF50+ PA+++
UV 아쿠아 리치 워터리 에센스 선크림 SPF50+ PA++++
스킨 릴리프 UV 프로텍터 SPF50+ PA+++
선 솔루션 프레쉬 선 에센스 IRF20 SPF50+ PA++++
유브이 퍼펙션 하이드라썬 SPF50+ PA+++
트루케어 논나노 논코메도 무기자차 선크림 SPF48 PA+++
블랙 티 올 데이 아쿠아 선 젤 SPF30 PA++
에브리 선데이
아임 쏘리 포 마이 스킨 데일리 카밍 선 크림 SPF50+ PA++++
선크림 스테이 선 세이프 SPF50+ PA++++
더마 디펜스 선 플루이드


In [472]:
dataSet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1923 entries, 0 to 1922
Data columns (total 4 columns):
cosmeticId    1923 non-null int64
rate          1923 non-null float64
review        1923 non-null object
type          1923 non-null int64
dtypes: float64(1), int64(2), object(1)
memory usage: 60.2+ KB
